In [1]:
pip install tensorflow-datasets


Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [5]:

import os
import numpy as np
import seaborn as sns
import tensorflow as tf
#import tensorflow_datasets as tdfs

In [6]:
#import tensorflow_datasets as tfds
cassava_data = tfds.load("cassava")
cassava_train, cassava_test = cassava_data["train"], cassava_data["test"]
assert isinstance(cassava_train, tf.data.Dataset)

NameError: name 'tfds' is not defined

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import array_to_img, img_to_array, load_img
import numpy as np
import os

from keras import models
from keras import layers
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dense
from sklearn import preprocessing 
from sklearn.preprocessing import StandardScaler, LabelBinarizer

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt
import PIL.Image as Image
import os
import tensorflow_addons as tfa

In [ ]:
data_santa_dir = 'data/santa/'
data_not_santa_dir = 'data/not_santa/'
imgs_santa = [file for file in os.listdir(data_santa_dir) if file.endswith('.jpg')]

In [ ]:
train_dir = 'cats_dogs_downsampled/train'
validation_dir = 'cats_dogs_downsampled/val/'
test_dir = 'cats_dogs_downsampled/test/' 

In [ ]:
train_data_dir = 'data/train'
test_data_dir = 'data/validation'

In [12]:
from zipfile import ZipFile
  
# loading the temp.zip and creating a zip object
with ZipFile("data/cassava-disease.zip/train.zip", 'r') as zObject:
  
    # Extracting all the members of the zip 
    # into a specific location.
    zObject.extractall(
        path='./')

with ZipFile("data/cassava-disease.zip/test.zip", 'r') as zObject:
  
    # Extracting all the members of the zip 
    # into a specific location.
    zObject.extractall(
        path='./')



FileNotFoundError: [Errno 2] No such file or directory: 'data/cassava-disease.zip/train.zip'

In [27]:
print(os.listdir("data"))
#print(os.listdir("../data/train/"))

['cassava-disease.zip']


In [33]:
os.listdir('./data/cassava-disease.zip')

NotADirectoryError: [WinError 267] The directory name is invalid: './data/cassava-disease.zip'

In [40]:
os.listdir('./data')

['cassava-disease.zip',
 'extraimages.zip',
 'random.txt',
 'sample_submission_file.csv',
 'test.zip',
 'train.zip']

In [43]:
from zipfile import ZipFile
  
# loading the temp.zip and creating a zip object
with ZipFile("data/cassava-disease.zip", 'r') as zObject:
  
    # Extracting all the members of the zip 
    # into a specific location.
    zObject.extractall(
        path='./data')

In [13]:
import os
for dirname, _, filenames in os.walk('./train'):
    for filename in filenames:
        Id.append(os.path.join(dirname, filename))
Id[:5]

NameError: name 'Id' is not defined

In [ ]:
!unzip /kaggle/input/cassava-disease/extraimages.zip -d  /kaggle/working/ 
!unzip /kaggle/input/cassava-disease/test.zip -d /kaggle/working/
!unzip /kaggle/input/cassava-disease/train.zip -d /kaggle/working/

In [44]:

import os
print(os.listdir("./data"))


['cassava-disease.zip', 'extraimages.zip', 'random.txt', 'sample_submission_file.csv', 'test.zip', 'train.zip']
